In [1]:
%matplotlib inline
import itertools
from time import time
import csv


import matplotlib.pyplot as plt
import numpy as np
from kifmm_py import KiFmm

# Single Precision - BLAS Field Translations

In [83]:
surface_diff_vec = [0, 1, 2]
svd_threshold_vec = [None, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 2e-2]
depth_vec = [4, 5]
expansion_order_vec = [3, 4, 5, 6]

parameters = list(itertools.product(surface_diff_vec, svd_threshold_vec, depth_vec, expansion_order_vec))

In [ ]:
np.random.seed(0)

dim = 3
dtype = np.float32
ctype = np.complex64

# Set FMM Parameters
n_vec = 1
n_crit = None
depth = 3
n_sources = 1000000
n_targets = 1000000
kernel = "laplace" 
field_translation = "blas"  
kernel_eval_type = (
    "eval"  
)

sources = np.reshape(
    np.random.rand(n_sources * dim), (n_sources, dim)
).astype(dtype)
targets = np.reshape(
    np.random.rand(n_targets * dim), (n_targets, dim)
).astype(dtype)
charges = np.reshape(
    np.random.rand(n_sources * n_vec), (n_sources, n_vec)
).astype(dtype)

rel_error = []
times = []
setup = []

for (i, (surface_diff, svd_threshold, depth, expansion_order)) in enumerate(parameters):
    tmp = [expansion_order]*(depth+1)

    s = time()
    fmm = KiFmm(
        tmp,
        sources,
        targets,
        charges,
        kernel_eval_type,
        kernel,
        field_translation,
        prune_empty=True,
        timed=True,
        svd_threshold=svd_threshold,
        surface_diff=surface_diff,
        depth=depth
    )
    setup.append(time()-s)

    s = time()
    fmm.evaluate()
    times.append(time()-s)
    print(f"Computing: {i+1}/{len(parameters)} Eval Time: {times[i]}")
        
    leaf = fmm.target_leaves[1]
    found = fmm.potentials(leaf)[0]
    targets_leaf = fmm.target_coordinates(leaf)
    expected = fmm.evaluate_kernel(sources, targets_leaf, charges)

    relative_error = np.abs(expected-found)/expected

    max_relative_error = np.max(relative_error)
    min_relative_error = np.min(relative_error)
    mean_relative_error = np.mean(relative_error)

    rel_error.append((min_relative_error, mean_relative_error, max_relative_error))

Computing: 1/216 Eval Time: 0.5378351211547852
Computing: 2/216 Eval Time: 0.5659511089324951
Computing: 3/216 Eval Time: 0.5799570083618164
Computing: 4/216 Eval Time: 0.6013541221618652
Computing: 5/216 Eval Time: 0.25028109550476074
Computing: 6/216 Eval Time: 0.35209107398986816
Computing: 7/216 Eval Time: 0.5447139739990234
Computing: 8/216 Eval Time: 0.8796911239624023
Computing: 9/216 Eval Time: 0.5351998805999756
Computing: 10/216 Eval Time: 0.5595970153808594
Computing: 11/216 Eval Time: 0.5711607933044434
Computing: 12/216 Eval Time: 0.631903886795044
Computing: 13/216 Eval Time: 0.2374098300933838
Computing: 14/216 Eval Time: 0.33855581283569336
Computing: 15/216 Eval Time: 0.5523383617401123
Computing: 16/216 Eval Time: 0.8616049289703369
Computing: 17/216 Eval Time: 0.5387027263641357
Computing: 18/216 Eval Time: 0.5491340160369873
Computing: 19/216 Eval Time: 0.570084810256958
Computing: 20/216 Eval Time: 0.610076904296875
Computing: 21/216 Eval Time: 0.23615097999572754


In [2]:
# with open('accuracy_single_precision_blas.csv', mode='w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(['surface_diff', 'svd_threshold', 'depth', 'expansion_order', 'time', 'min_rel_err', 'mean_rel_err', 'max_rel_err'])
#     for row in zip(parameters, times, rel_error):
#         writer.writerow([row[0][0], row[0][1], row[0][2], row[0][3], row[1], row[2][0], row[2][1], row[2][2]])

# Single Precision - FFT Field Translations

In [2]:
depth_vec = [4, 5]
expansion_order_vec = [3, 4, 5, 6]

parameters = list(itertools.product(depth_vec, expansion_order_vec))

np.random.seed(0)

dim = 3
dtype = np.float32
ctype = np.complex64

# Set FMM Parameters
n_vec = 1
n_crit = None
n_sources = 1000000
n_targets = 1000000
kernel = "laplace" 
field_translation = "fft"  
kernel_eval_type = (
    "eval"  
)

sources = np.reshape(
    np.random.rand(n_sources * dim), (n_sources, dim)
).astype(dtype)
targets = np.reshape(
    np.random.rand(n_targets * dim), (n_targets, dim)
).astype(dtype)
charges = np.reshape(
    np.random.rand(n_sources * n_vec), (n_sources, n_vec)
).astype(dtype)

rel_error = []
times = []
setup = []


for (i, (depth, expansion_order)) in enumerate(parameters):
    tmp = [expansion_order]*(depth+1)
    s = time()
    fmm = KiFmm(
        tmp,
        sources,
        targets,
        charges,
        kernel_eval_type,
        kernel,
        field_translation,
        prune_empty=True,
        timed=True,
        depth=depth
    )
    setup.append(time()-s)

    s = time()
    fmm.evaluate()
    times.append(time()-s)
    print(f"Computing: {i+1}/{len(parameters)} Eval Time: {times[i]}")
        
    leaf = fmm.target_leaves[1]
    found = fmm.potentials(leaf)[0]
    targets_leaf = fmm.target_coordinates(leaf)
    expected = fmm.evaluate_kernel(sources, targets_leaf, charges)

    relative_error = np.abs(expected-found)/expected

    max_relative_error = np.max(relative_error)
    min_relative_error = np.min(relative_error)
    mean_relative_error = np.mean(relative_error)

    rel_error.append((min_relative_error, mean_relative_error, max_relative_error))

Computing: 1/8 Eval Time: 0.5398528575897217
Computing: 2/8 Eval Time: 0.5751571655273438
Computing: 3/8 Eval Time: 0.5838930606842041
Computing: 4/8 Eval Time: 0.6210978031158447
Computing: 5/8 Eval Time: 0.18736839294433594
Computing: 6/8 Eval Time: 0.30138278007507324
Computing: 7/8 Eval Time: 0.47861385345458984
Computing: 8/8 Eval Time: 0.6416518688201904


In [4]:
# with open('accuracy_single_precision_fft.csv', mode='w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(['depth', 'expansion_order', 'time', 'min_rel_err', 'mean_rel_err', 'max_rel_err'])
#     for row in zip(parameters, times, rel_error):
#         writer.writerow([row[0][0], row[0][1], row[1], row[2][0], row[2][1], row[2][2]])